## Model 1: Entry and exit rules for backtesting

**Model 1:** EMA50 crossover entry and EMA20 downtrend signal exit.

In [18]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
from datetime import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import sys
import pandas as pd
import numpy as np
import glob
import nbimporter
import gzip
from modelPEG import main


In [19]:
# Call the main function and store the returned DataFrame
df = main()

File loaded successfully!


In [20]:
# Show dataframe
print(df.head())

       symbol  close   open   high    low   volume   timestamp  volatility  \
318      AAOI   5.27   5.20   5.30   5.08   478600  2022-01-03         NaN   
507822   PACB  20.90  20.50  21.01  19.51  2428600  2022-01-03         NaN   
505755    OSW  10.50  10.08  10.73  10.08   252000  2022-01-03         NaN   
505066   OSPN  17.23  16.99  17.26  16.74   126700  2022-01-03         NaN   
503688   ORLA   3.88   3.81   3.88   3.64    43300  2022-01-03         NaN   

        trailingStop  exceptionalCandleSize  ema20  ema50  openHigher  \
318              NaN                      0   5.27   5.27           0   
507822           NaN                      0  20.90  20.90           0   
505755           NaN                      0  10.50  10.50           0   
505066           NaN                      0  17.23  17.23           0   
503688           NaN                      0   3.88   3.88           0   

        averageVolume  strongVolume  strongVolume6MoPrior  accVolume  uptrend  \
318        

In [32]:
# Example DataFrame setup
# df_test = pd.DataFrame({
#     'timestamp': pd.to_datetime([1,2,3,4,5,6,7,8,9,10], unit='D', origin='2022-01-01'),  # sample dates
#     'symbol': ['AA','AA','AA','AA','AA','BB','BB','BB','BB','BB'],  
#     'close': [10,9,11,11,9,100,110,120,125,150],
#     'ema50': [9,10,10.5,10,10,95,115,110,120,130],
#     'pegAlert': [1,0,0,0,0,1,0,0,0,0],
#     'trailingStop': [11,11,12,12,8,101,111,121,126,145]
# })

In [74]:
# Example DataFrame setup
# df_test = pd.DataFrame({
#     'timestamp': pd.to_datetime([1,2,3,4,5], unit='D', origin='2022-01-01'),  # sample dates
#     'symbol': ['BB','BB','BB','BB','BB'],  
#     'close': [100,110,120,130,150],
#     'ema50': [95,115,110,120,130],
#     'pegAlert': [1,0,0,0,0]
# })

In [33]:
def calculate_cross_below_ema50(df):    
    df['crossBelowEma50'] = 0  # Initialize the Entry column with 0

    # Reset index to ensure consecutive indexing within each symbol group
    df = df.reset_index(drop=True)

    # Iterate over rows where alert is 1
    for i, row in df[df['pegAlert'] == 1].iterrows():
        # Define the 90-row window after the alert
        end_index = min(i + 90, len(df))  # Ensure we don't go out of range
        for j in range(i + 1, end_index):
            # Check if 'close' is below 'ema50' within the 90-row window
            if df.at[j, 'close'] < df.at[j, 'ema50']:
                df.at[j, 'crossBelowEma50'] = 1  # Set 'Entry' to 1 in the row where condition is met
                break  # Stop searching within the 90-row window once condition is satisfied
    return df


In [34]:
def calculate_entry(df):    
    # Initialize the entryPos column with 0
    df['entryPos'] = 0

    # Reset index to ensure consecutive indexing within each symbol group
    df = df.reset_index(drop=True)  

    # Iterate over rows where 'crossBelowEma50' is 1
    for i, row in df[df['crossBelowEma50'] == 1].iterrows():
        # Define the 30-row window after the alert
        end_index = min(i + 30, len(df))  # Ensure we don't go out of range
        for j in range(i + 1, len(df)):
            # Check if 'close' is above 'ema50' within the 30-row window
            if df.at[j, 'close'] > df.at[j, 'ema50']:
                # The immediate next row (j + 1) remains 0
                delayed_index_2 = j + 1  # Check 2 rows after the condition is met
                
                # Check for the condition two rows after j
                if delayed_index_2 < len(df) and df.at[delayed_index_2, 'close'] > df.at[delayed_index_2, 'ema50']:
                    df.at[delayed_index_2, 'entryPos'] = 1  # Set entryPos to 1 for j + 2
                
                break  # Stop searching within the 30-row window once the first condition is satisfied
    return df


In [35]:
def calculate_cross_below_ema50_for_symbols(df, symbols):
    # Initialize an empty list to hold DataFrames for each symbol
    df_list = []

    for symbol in symbols:
        # Filter the DataFrame for the current symbol
        df_symbol = df[df['symbol'] == symbol].copy()
        
        # Call the calculation function for the filtered symbol DataFrame
        df_symbol = calculate_cross_below_ema50(df_symbol)
        
        # Append the processed DataFrame to the list
        df_list.append(df_symbol)

    # Concatenate all symbol DataFrames into one final DataFrame
    df_final = pd.concat(df_list, ignore_index=True)
    
    return df_final

In [36]:
# List of symbols from dataframe
symbols_list = df['symbol'].unique()
print(symbols_list)

['AA' 'BB']


In [37]:

df = calculate_cross_below_ema50_for_symbols(df, symbols_list)
print(df)

   timestamp symbol  close  ema50  pegAlert  trailingStop  crossBelowEma50
0 2022-01-02     AA     10    9.0         1            11                0
1 2022-01-03     AA      9   10.0         0            11                1
2 2022-01-04     AA     11   10.5         0            12                0
3 2022-01-05     AA     11   10.0         0            12                0
4 2022-01-06     AA      9   10.0         0             8                0
5 2022-01-07     BB    100   95.0         1           101                0
6 2022-01-08     BB    110  115.0         0           111                1
7 2022-01-09     BB    120  110.0         0           121                0
8 2022-01-10     BB    125  120.0         0           126                0
9 2022-01-11     BB    150  130.0         0           145                0


In [38]:
def calculate_entry_for_symbols(df, symbols):
    # Initialize an empty list to hold DataFrames for each symbol
    df_list = []

    for symbol in symbols:
        # Filter the DataFrame for the current symbol
        df_symbol = df[df['symbol'] == symbol].copy()
        
        # Call the calculation function for the filtered symbol DataFrame
        df_symbol = calculate_entry(df_symbol)
        
        # Append the processed DataFrame to the list
        df_list.append(df_symbol)

    # Concatenate all symbol DataFrames into one final DataFrame
    df_final = pd.concat(df_list, ignore_index=True)
    
    return df_final

In [39]:
df = calculate_entry_for_symbols(df, symbols_list)
print(df)

   timestamp symbol  close  ema50  pegAlert  trailingStop  crossBelowEma50  \
0 2022-01-02     AA     10    9.0         1            11                0   
1 2022-01-03     AA      9   10.0         0            11                1   
2 2022-01-04     AA     11   10.5         0            12                0   
3 2022-01-05     AA     11   10.0         0            12                0   
4 2022-01-06     AA      9   10.0         0             8                0   
5 2022-01-07     BB    100   95.0         1           101                0   
6 2022-01-08     BB    110  115.0         0           111                1   
7 2022-01-09     BB    120  110.0         0           121                0   
8 2022-01-10     BB    125  120.0         0           126                0   
9 2022-01-11     BB    150  130.0         0           145                0   

   entryPos  
0         0  
1         0  
2         0  
3         1  
4         0  
5         0  
6         0  
7         0  
8         1  
9

In [40]:
def calculate_exit(df):    
    df['exitPos'] = 0  # Initialize the Entry column with 0

    # Reset index to ensure consecutive indexing within each symbol group
    df = df.reset_index(drop=True)

    # Iterate over rows where alert is 1
    for i, row in df[df['entryPos'] == 1].iterrows():
        # Define the max df range.
        for j in range(i + 1, len(df)):
            # Check if 'trailingStop' is higher than 'low' within dataframe.
            if df.at[j, 'trailingStop'] >= df.at[j, 'low']:
                df.at[j, 'exitPos'] = 1  # Set 'exitPos' to 1 in the row where condition is met
                break  # Stop searching within the df once condition is satisfied
    return df

In [41]:
def calculate_exit_for_symbols(df, symbols):
    # Initialize an empty list to hold DataFrames for each symbol
    df_list = []

    for symbol in symbols:
        # Filter the DataFrame for the current symbol
        df_symbol = df[df['symbol'] == symbol].copy()
        
        # Call the calculation function for the filtered symbol DataFrame
        df_symbol = calculate_exit(df_symbol)
        
        # Append the processed DataFrame to the list
        df_list.append(df_symbol)

    # Concatenate all symbol DataFrames into one final DataFrame
    df_final = pd.concat(df_list, ignore_index=True)
    
    return df_final

In [42]:
df = calculate_exit_for_symbols(df, symbols_list)
print(df)

   timestamp symbol  close  ema50  pegAlert  trailingStop  crossBelowEma50  \
0 2022-01-02     AA     10    9.0         1            11                0   
1 2022-01-03     AA      9   10.0         0            11                1   
2 2022-01-04     AA     11   10.5         0            12                0   
3 2022-01-05     AA     11   10.0         0            12                0   
4 2022-01-06     AA      9   10.0         0             8                0   
5 2022-01-07     BB    100   95.0         1           101                0   
6 2022-01-08     BB    110  115.0         0           111                1   
7 2022-01-09     BB    120  110.0         0           121                0   
8 2022-01-10     BB    125  120.0         0           126                0   
9 2022-01-11     BB    150  130.0         0           145                0   

   entryPos  exitPos  
0         0        0  
1         0        0  
2         0        0  
3         1        0  
4         0        1  
5  